# U-Net Experiment 2: Exploring benefit of tile-overlapping

### 0. Select Python version

In [1]:
!sudo apt-get install python3.10
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --config python3

### 1. Loading + Preparing Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')
! ls
%cd drive/MyDrive/MachineLearning/Geospatial_ML

In [4]:
# ! pip install -r ../requirements.txt
import numpy as np
import os
root = "../data_colab/256_256"
# root = "../data/experiment1"

y_train  = np.load(os.path.join(root,'2022_06_20.npz'))['y_mask']
x_train  = np.load(os.path.join(root,'2022_06_20.npz'))['x_input']

y_val = np.load(os.path.join(root,'2022_07_10.npz'))['y_mask']
x_val = np.load(os.path.join(root,'2022_07_10.npz'))['x_input']

y_test = np.load(os.path.join(root, '2022_07_25.npz'))['y_mask']
x_test = np.load(os.path.join(root, '2022_07_25.npz'))['x_input']


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


### 2. Set absolute path to import functions

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### 3. Normalizing

In [5]:
from experiments.helpers import normalizing

x_train, y_train = normalizing(x_train, y_train)
X_val, y_val = normalizing(x_val, y_val)
x_test, y_test = normalizing(x_test, y_test)

2023-03-31 18:02:00.031617: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


y shape:  (1251, 256, 256)
Normalizing data...
y one hot shape:  (1251, 256, 256, 3)
y shape:  (1251, 256, 256)
Normalizing data...
y one hot shape:  (1251, 256, 256, 3)
y shape:  (1251, 256, 256)
Normalizing data...
y one hot shape:  (1251, 256, 256, 3)


### 4. Compiling the model

In [6]:
from models.unet_model import unet_2d
model = unet_2d(input_shape=(256, 256, 5), num_classes=3)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 5  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  2944        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

### 5. Execute trainings + saving results

In [7]:
tile_size = 256
step_size = 200
saving_path = 'experiment_1'
training_dates = '2022_06_20'
validation_dates = '2022_07_10'
testing_dates = '2022_07_25'
all_metrics_test = []
all_metrics_val = []
all_metrics_train = []

In [8]:
from experiments.helpers import define_model, train_model, save_model, save_model_history, predict_model, evaluate_predictions, save_metrics

# Define model
model = define_model((256, 256, 5), 3)

for count in range(0, 10):
    print(f'Start training number {count}')
    # Train model
    history = train_model(model, x_train, y_train, x_val, y_val, epochs=100)
    print('training completed')
    # Save model
    model_name = f'{tile_size}_{step_size}_run_{count}'
    save_model(model, model_name, saving_path)
    print('saving model completed')
    # Save model history
    save_model_history(history, model_name, saving_path)
    print('saving history completed')
    # Predict 
    pred_train, pred_val, pred_test = predict_model(model, x_train, x_val, x_test)
    print('prediction completed')
    # Evaluate predictions
    metrics_dict = evaluate_predictions(x_train, x_val, x_test, y_train, y_val, y_test, pred_train, pred_val, pred_test, training_dates, validation_dates, testing_dates, tile_size, step_size, count)
    print('evaluation completed')
    # Save metrics
    save_metrics(metrics_dict, model_name, saving_path)
    print('saving metrics completed')
    all_metrics_test.append(metrics_dict['metrics_test'])
    all_metrics_val.append(metrics_dict['metrics_val'])
    all_metrics_train.append(metrics_dict['metrics_train'])

Start training number 0
Epoch 1/100
 1/40 [..............................] - ETA: 1:25:30 - loss: 1.0915 - accuracy: 0.4066

### 6. Results

In [ ]:
for idx, metric in enumerate(all_metrics_test):
  print(f'========= RUN {idx + 1} ============')
  print(f'TEST DATA')
  metric.print_metrics()
  print()
  print(f'VALIDATION DATA')
  all_metrics_val[idx].print_metrics()
  print(f'TRAINING DATA')
  all_metrics_train[idx].print_metrics()
  print()
  print()

In [ ]:
from experiments.helpers import get_mean_jaccard

print('============ TEST DATA ===================')
get_mean_jaccard(all_metrics_test)
print()

print('============ VALIDATION DATA ===================')
get_mean_jaccard(all_metrics_val)
print()

print('============ TRAINING DATA ===================')
get_mean_jaccard(all_metrics_train)
print()